In [11]:
import FinanceDataReader as fdr
import os
import openpyxl
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
from pykrx import stock
import pandas_datareader.data as pdr
from datetime import datetime, timedelta

In [12]:
# Define the directory containing the Excel files
folder_path = './kospi/kospi_component'  # Adjust the path as needed

# Initialize an empty DataFrame to store the results
kospi = pd.DataFrame()

In [13]:
# def read_excel_files(folder_path):
#     # Create an empty list to store index codes
#     idx_code = list()
    
#     # Loop through all files in the specified folder
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".xlsx") or filename.endswith(".xls"):
#             file_path = os.path.join(folder_path, filename)
#             # Read the Excel file
#             df = pd.read_excel(file_path, dtype=str)
            
#             # Extract the '종목코드' column if it exists
#             if '종목코드' in df.columns:
#                 idx_code.extend(df['종목코드'].tolist())
    
#     # Remove duplicates by converting the list to a set
#     idx_code = set(idx_code)
    
#     # Convert the set back to a list and create a DataFrame
#     df_result = pd.DataFrame([list(idx_code)])
    
#     return df_result

In [14]:
# result_df = read_excel_files(folder_path)

c:\Users\zin\Developer\RankingMeetsIndexTracking\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\zin\Developer\RankingMeetsIndexTracking\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\zin\Developer\RankingMeetsIndexTracking\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\zin\Developer\RankingMeetsIndexTracking\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

In [ ]:
# def get_unique_tickers(start_date, end_date, market="KOSPI"):
#     # Convert string dates to datetime objects
#     start = datetime.strptime(start_date, '%Y-%m-%d')
#     end = datetime.strptime(end_date, '%Y-%m-%d')
    
#     # Initialize an empty list to store unique tickers
#     unique_tickers = list()
    
#     # Iterate through each day in the date range
#     current_date = start
#     while current_date <= end:
#         # Get the list of tickers for the current date
#         tickers = stock.get_market_ticker_list(current_date.strftime('%Y-%m-%d'), market=market)
#         # Add tickers to the set (sets automatically handle duplicates)
#         unique_tickers.append(tickers)
#         # Move to the next day
#         current_date += timedelta(days=1)
    
#     unique_tickers = set(unique_tickers)
#     # Convert the set to a DataFrame
#     tickers_df = pd.DataFrame(list(unique_tickers))
#     tickers_df.columns = tickers_df.iloc[0]
#     tickers_df = tickers_df[1:].reset_index(drop=True)
#     return tickers_df

# # Define the date range
# start_date = '2006-01-01'
# end_date = '2022-12-31'

# # Get the DataFrame of unique tickers
# unique_tickers_df = get_unique_tickers(start_date, end_date)

# # Display the DataFrame
# unique_tickers_df.head()

In [16]:
# date_range = pd.date_range(start='2006-01-01', end='2022-12-31', freq='D')
# # DataFrame 생성
# date_df = pd.DataFrame(date_range, columns=['date'])
# date_df['^KS11'] = np.nan

In [18]:
# kospi = pd.concat([date_df, result_df], axis=1)
# kospi.head()

,date,^KS11,093050,000040,077970,003550,052690,088790,072130,009070,...,000990,011000,018260,001570,172580,009410,008040,001970,002630,003220
0,2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2006-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2006-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2006-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Yahho finance에서 데이터 가져오기- 수정주가

In [ ]:
kospi = pd.read_csv('.\kospi\kospi_stock_data.csv', parse_dates=['date'], index_col='date')

In [19]:
unique_tickers = kospi.columns
unique_tickers

In [ ]:
def update_ticker_data(ticker, df, start_date='2006-01-01', end_date='2022-12-31'):
    # Append .KS to the ticker for yfinance
    ticker = str(ticker)
    ticker_ks = ticker + '.KS'
    try:
        # Fetch historical market data using yfinance
        ticker_data = yf.download(ticker_ks, start=start_date, end=end_date)
        ticker_data = ticker_data['Close']
        df[ticker] = ticker_data
    except Exception as e:
        print(f"Failed to fetch data for {ticker} using yfinance: {e}")
        try:
            # If yfinance fails, fetch data using pandas_datareader and naver
            ticker_data = pdr.DataReader(ticker, 'naver', start=start_date, end=end_date)
            ticker_data = ticker_data['Close']
            df[ticker] = ticker_data
        except Exception as e:
            print(f"Failed to fetch data for {ticker} using naver: {e}")
            return df
    return df

In [ ]:
# Update the dataframe with the closing prices for each unique ticker
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    kospi = update_ticker_data(ticker, kospi)

In [ ]:
kospi.head()

In [21]:
# Save the updated dataframe to a CSV file
output_file_path = '.\kospi\kospi_stock_data.csv'
kospi.to_csv(output_file_path, index=True)

In [ ]:
#모든 행이 NA면 제거(공휴일)
kospi = kospi.dropna(how='all')

In [ ]:
# 열을 검사하여 5열 연속 비어있는 열 제거
cols_to_drop = [col for col in kospi.columns if kospi[col].isna().sum() >= 5]
df_cleaned = kospi.drop(columns=cols_to_drop)

In [ ]:
# 선형 보간법을 사용하여 NaN 값 채우기
kospi = kospi.interpolate(method='linear')

In [ ]:
# Save the updated dataframe to a CSV file
output_file_path = '.\kospi\clean_kospi_stock_data.csv'
kospi.to_csv(output_file_path, index=True)